In [29]:
from gurobipy import *
import numpy as np

In [30]:
#Indices
t_array = np.arange(0,6)
m_array = np.arange(0,8)
k_array = np.arange(0, 20)
i_array = np.arange(0, 10)
j_array = np.arange(0, 3)

In [31]:
oi = Model()

In [59]:
#Variables
x = oi.addVars(((i,m) for i in i_array for m in m_array),vtype = GRB.BINARY)
e = oi.addVars(((k,i) for k in k_array for i in i_array),vtype = GRB.BINARY)
b = oi.addVars(((i) for i in i_array),vtype = GRB.BINARY)
y = oi.addVars(((k) for k in k_array),vtype = GRB.BINARY)
o = oi.addVars(((i,k) for i in i_array for k in k_array),vtype = GRB.CONTINUOUS)
w = oi.addVars(((i) for i in i_array),vtype = GRB.BINARY)
profit = oi.addVars(((i) for i in i_array),vtype = GRB.CONTINUOUS)

In [60]:
#Matrices
R_i =np.array([250, 1800, 1050, 3550, 550, 3600, 700, 3650, 1350, 2800])
et_tk = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
L = 2
p = 1500
H = 280
G_j = np.array([10000, 50000, 800000])
S_t = np.array([8000, 16000, 24000, 32000, 40000, 48000])
ub_ti = np.array([
    [2, 2, 2, 3, 3, 3, 3, 3, 2, 3],
    [3, 2, 3, 2, 3, 3, 3, 2, 2, 2],
    [2, 2, 2, 3, 2, 2, 2, 2, 3, 2],
    [3, 2, 3, 2, 2, 2, 3, 2, 3, 3],
    [2, 3, 2, 2, 2, 3, 3, 2, 3, 3],
    [2, 3, 3, 2, 3, 2, 2, 2, 3, 3]])
lb_ti = np.array([
    [1, 1, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 1, 0, 1, 1, 0, 0, 1, 1],
    [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]])
d_ij = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [0, 0, 1],
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 0, 1]])
a_im = np.array([
    [0, 0, 0, 0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1, 0, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1]])

In [61]:
#Constraints
for k in k_array:
    oi.addConstr(quicksum(e[k,i] for i in i_array)>=1)

In [62]:
for k in k_array:
    oi.addConstr(quicksum(e[k,i] for i in i_array)<=5)

In [63]:
for i in i_array:
    for t in t_array:
        oi.addConstr((quicksum(e[k,i] * et_tk[t,k] for k in k_array)<=ub_ti[t,i]))

In [64]:
for i in i_array:
    for t in t_array:
        oi.addConstr((quicksum(e[k,i] * et_tk[t,k] for k in k_array)>=lb_ti[t,i]))

In [65]:
for m in m_array:
    oi.addConstr(quicksum(x[i,m] for i in i_array)>=1)

In [66]:
for i in i_array:
    oi.addConstr(quicksum(x[i,m] for m in m_array)<=1)

In [67]:
oi.addConstr(quicksum(x[i,0] - x[i,1] for i in i_array)<=L)

<gurobi.Constr *Awaiting Model Update*>

In [68]:
oi.addConstr(quicksum(x[i,1] - x[i,0] for i in i_array)<=L)

<gurobi.Constr *Awaiting Model Update*>

In [69]:
for k in k_array:
    oi.addConstr(quicksum(o[i,k] for i in i_array)+y[k]*8*30<= H)

In [70]:
for k in k_array:
    oi.addConstr(y[k] <= quicksum(e[k,i] for i in i_array))

In [71]:
for k in k_array:
    oi.addConstr(10*y[k]>=quicksum(e[k,i] for i in i_array))

In [72]:
for k in k_array:
    for i in i_array:
        oi.addConstr(o[i,k]<=H*e[k,i])

In [73]:
for i in i_array:
    oi.addConstr(quicksum(x[i,m] for m in m_array)>=b[i])

In [74]:
for i in i_array:
    for m in m_array:
        oi.addConstr(x[i,m]<=a_im[i,m])

In [75]:
for i in i_array:
    oi.addConstr(quicksum(8*30*e[k,i] + o[i,k]*d_ij[i,2] for k in k_array) >= (R_i[i]*b[i]))

In [76]:
oi.addConstr(x[0,0]>=w[0])
oi.addConstr(x[1,0]>=w[0])
oi.addConstr(x[0,0]+x[1,0]-1<=w[0])
oi.addConstr(1-w[0]<=x[2,2])

<gurobi.Constr *Awaiting Model Update*>

In [77]:
for i in i_array:
    oi.addConstr((1-b[i])*d_ij[i,1]<=w[1])

In [78]:
for i in i_array:
    oi.addConstr(1-w[1]>=(1-b[i])*d_ij[i,2])

In [79]:
for i in i_array:
    for k in k_array:
        oi.addConstr(o[i,k] >= 0)

In [86]:
oi.addConstr(profit[0] == quicksum(d_ij[i,j]*b[i]*G_j[j] for i in i_array for j in j_array)-quicksum(p*(1-b[i])for i in i_array)-quicksum(e[k,i]*S_t[t]*et_tk[t,k] for i in i_array for t in t_array for k in k_array)-quicksum(o[i,k]*50 for i in i_array for k in k_array))

<gurobi.Constr *Awaiting Model Update*>

In [87]:
oi.setObjective(quicksum(b[i] for i in i_array),GRB.MAXIMIZE)

In [88]:
oi.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1529 rows, 1050 columns and 5923 nonzeros
Model fingerprint: 0x2b9f9704
Variable types: 410 continuous, 640 integer (640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]

MIP start from previous solve produced solution with objective 9 (0.01s)
Loaded MIP start from previous solve with objective 9

Presolve removed 1300 rows and 759 columns
Presolve time: 0.01s
Presolved: 229 rows, 291 columns, 1500 nonzeros
Variable types: 80 continuous, 211 integer (211 binary)

Root relaxation: infeasible, 153 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

In [90]:
profit[0]

<gurobi.Var C1040 (value 514500.0)>

In [91]:
o

{(0, 0): <gurobi.Var C830 (value 0.0)>,
 (0, 1): <gurobi.Var C831 (value 0.0)>,
 (0, 2): <gurobi.Var C832 (value 0.0)>,
 (0, 3): <gurobi.Var C833 (value 0.0)>,
 (0, 4): <gurobi.Var C834 (value 0.0)>,
 (0, 5): <gurobi.Var C835 (value 0.0)>,
 (0, 6): <gurobi.Var C836 (value 0.0)>,
 (0, 7): <gurobi.Var C837 (value 0.0)>,
 (0, 8): <gurobi.Var C838 (value 0.0)>,
 (0, 9): <gurobi.Var C839 (value 0.0)>,
 (0, 10): <gurobi.Var C840 (value 0.0)>,
 (0, 11): <gurobi.Var C841 (value 0.0)>,
 (0, 12): <gurobi.Var C842 (value 0.0)>,
 (0, 13): <gurobi.Var C843 (value 0.0)>,
 (0, 14): <gurobi.Var C844 (value 0.0)>,
 (0, 15): <gurobi.Var C845 (value 0.0)>,
 (0, 16): <gurobi.Var C846 (value 0.0)>,
 (0, 17): <gurobi.Var C847 (value 0.0)>,
 (0, 18): <gurobi.Var C848 (value 0.0)>,
 (0, 19): <gurobi.Var C849 (value 0.0)>,
 (1, 0): <gurobi.Var C850 (value 0.0)>,
 (1, 1): <gurobi.Var C851 (value 0.0)>,
 (1, 2): <gurobi.Var C852 (value 0.0)>,
 (1, 3): <gurobi.Var C853 (value 0.0)>,
 (1, 4): <gurobi.Var C854 (val

In [92]:
e

{(0, 0): <gurobi.Var C600 (value 0.0)>,
 (0, 1): <gurobi.Var C601 (value 1.0)>,
 (0, 2): <gurobi.Var C602 (value 0.0)>,
 (0, 3): <gurobi.Var C603 (value 1.0)>,
 (0, 4): <gurobi.Var C604 (value 0.0)>,
 (0, 5): <gurobi.Var C605 (value 1.0)>,
 (0, 6): <gurobi.Var C606 (value 0.0)>,
 (0, 7): <gurobi.Var C607 (value 1.0)>,
 (0, 8): <gurobi.Var C608 (value 0.0)>,
 (0, 9): <gurobi.Var C609 (value 1.0)>,
 (1, 0): <gurobi.Var C610 (value 0.0)>,
 (1, 1): <gurobi.Var C611 (value 0.0)>,
 (1, 2): <gurobi.Var C612 (value 0.0)>,
 (1, 3): <gurobi.Var C613 (value 0.0)>,
 (1, 4): <gurobi.Var C614 (value 1.0)>,
 (1, 5): <gurobi.Var C615 (value 1.0)>,
 (1, 6): <gurobi.Var C616 (value 0.0)>,
 (1, 7): <gurobi.Var C617 (value 1.0)>,
 (1, 8): <gurobi.Var C618 (value 1.0)>,
 (1, 9): <gurobi.Var C619 (value 0.0)>,
 (2, 0): <gurobi.Var C620 (value 0.0)>,
 (2, 1): <gurobi.Var C621 (value 1.0)>,
 (2, 2): <gurobi.Var C622 (value 0.0)>,
 (2, 3): <gurobi.Var C623 (value 1.0)>,
 (2, 4): <gurobi.Var C624 (value 1.0)>,
